# Notebook 03: Modeling & Evaluation

Notebook này tập trung vào việc **xây dựng, huấn luyện và đánh giá mô hình Collaborative Filtering** cho hệ thống gợi ý sản phẩm từ dữ liệu Amazon Beauty Ratings.

Trong các notebook trước, quá trình Data Exploration đã chỉ ra rằng:

- Ma trận user–item **rất thưa** (sparsity cực cao).  
- Phần lớn người dùng có **rất ít đánh giá** => Cold Start problem nghiêm trọng.  
- Phần lớn sản phẩm cũng chỉ có 1–2 đánh giá => Item Cold Start cũng tồn tại.
- Điều này khiến các mô hình Memory-based CF như **User-based / Item-based CF** hoạt động kém hiệu quả vì:  
  - Không có đủ neighbors tương đồng.  
  - Similarity-based methods dễ nhiễu khi dữ liệu quá thưa.

**Matrix Factorization giúp giảm chiều và “học” latent factors**, khắc phục hiệu quả sparsity. Do đó, mô hình Matrix Factorization là lựa chọn tốt cho hệ thống gợi ý này.

## 1: Import Libraries & Setup

In [1]:
import sys
sys.path.append('../src')

from pathlib import Path
import numpy as np

from models import (
    load_user_feature_dict,
    load_item_feature_dict,
    group_shuffle_split,
    PureNumpyMF,
    PureNumpyMFConfig,
    regression_metrics,
)

print(' NumPy Matrix Factorization pipeline ready!')


 NumPy Matrix Factorization pipeline ready!


## 2: Load Data

In [2]:
CLEAN_DATA_PATH = Path('../data/processed/ratings_Beauty_processed_clean.npz')
clean_npz = np.load(CLEAN_DATA_PATH)
clean_data = clean_npz['data'] if 'data' in clean_npz else clean_npz[list(clean_npz.files)[0]]

user_ids = clean_data['UserId']
item_ids = clean_data['ProductId']
ratings = clean_data['Rating'].astype(np.float32)

print(f'Source file : {CLEAN_DATA_PATH}')
print(f'Ratings     : {len(ratings):,}')
print(f'Unique users: {np.unique(user_ids).size:,}')
print(f'Unique items: {np.unique(item_ids).size:,}')
print(f'Mean rating : {ratings.mean():.4f}')


Source file : ..\data\processed\ratings_Beauty_processed_clean.npz
Ratings     : 2,023,070
Unique users: 1,210,271
Unique items: 249,274
Mean rating : 4.1490


### 2b: Load engineered user & item features


In [3]:
USER_FEATURE_PATH = Path('../data/processed/user_features.npz')
ITEM_FEATURE_PATH = Path('../data/processed/product_features.npz')

user_feature_dict, user_feat_dim = load_user_feature_dict(str(USER_FEATURE_PATH))
item_feature_dict, item_feat_dim = load_item_feature_dict(str(ITEM_FEATURE_PATH))

print('Feature stats:')
print(f'  User features   : {len(user_feature_dict):,} users, dim = {user_feat_dim}')
print(f'  Product features: {len(item_feature_dict):,} items, dim = {item_feat_dim}')


Feature stats:
  User features   : 1,210,271 users, dim = 3
  Product features: 249,274 items, dim = 3


## 3: Train-Test Split

In [4]:
train_idx, test_idx = group_shuffle_split(user_ids, test_ratio=0.2, random_state=42)

train_users = user_ids[train_idx]
train_items = item_ids[train_idx]
train_ratings = ratings[train_idx]

test_users = user_ids[test_idx]
test_items = item_ids[test_idx]
test_ratings = ratings[test_idx]

print(f'Train size: {train_users.size:,} ratings')
print(f'Test size : {test_users.size:,} ratings')


Train size: 1,620,121 ratings
Test size : 402,949 ratings


## 4: Model Training
- Model: PureNumpyMF (chi tiết trong models.py)
- Config: use_user_features=True, use_item_features=True - tích hợp feature engineering từ Notebook 01
- SGD-based Matrix Factorization: sử dụng Stochastic Gradient Descent để tối ưu hóa hàm mất mát, implement bằng numpy thuần, có regularization để giảm overfitting.

In [5]:
config = PureNumpyMFConfig(use_user_features=True, use_item_features=True)  
np_mf = PureNumpyMF(config)

train_metrics = np_mf.fit(
    train_users,
    train_items,
    train_ratings,
    user_feature_dict=user_feature_dict,
    item_feature_dict=item_feature_dict,
)
print(f' Training RMSE: {train_metrics["RMSE"]:.4f}, MAE: {train_metrics["MAE"]:.4f}')


 Training RMSE: 1.1319, MAE: 0.6031


In [6]:
print('TEST METRICS:')
test_preds = np_mf.predict(test_users, test_items)
test_metrics = regression_metrics(test_ratings, test_preds)
print(f' Test RMSE: {test_metrics["RMSE"]:.4f}, MAE: {test_metrics["MAE"]:.4f}')

TEST METRICS:
 Test RMSE: 1.3142, MAE: 1.0520


In [7]:
sample_user = train_users[0]
rated_mask = train_users == sample_user
rated_items = set(train_items[rated_mask])

recommendations = np_mf.recommend(sample_user, n=10, exclude_items=rated_items)

print(f'Sample user: {sample_user}')
print('Already rated:', len(rated_items))
print('Top 10 recommendations:')
for row in recommendations:
    print(f"  {row['product_id']}: {row['predicted_rating']:.3f}")


Sample user: A39HTATAQ9V7YF
Already rated: 4
Top 10 recommendations:
  B0068Y6CA4: 5.000
  B004Z40048: 5.000
  B007BLN17K: 5.000
  B00538TSMU: 5.000
  B0048O2R1E: 5.000
  B008LDFU26: 5.000
  B009T47YZ2: 5.000
  B0009V1YR8: 5.000
  B00EVLHLK6: 4.980
  B00875D5X4: 4.970


### Phân tích và đánh giá kết quả
*Kết quả từ lần train gần nhất*

| Metric | Training | Test |
|--------|----------|------|
| **RMSE** | **1.1319** | **1.3142** |
| **MAE**  | **0.6031** | **1.0520** |

#### **1. Sai số tăng khá nhiều từ Training -> Test**
- RMSE tăng từ **1.13 -> 1.31**  
- MAE tăng từ **0.6 -> 1.05**

Điều này cho thấy mô hình **học tốt trên dữ liệu đã thấy**, nhưng **khả năng tổng quát hoá trên dữ liệu mới còn hạn chế**.

Đây là hiện tượng phổ biến trong **Matrix Factorization** khi:

- Số lượng rating mỗi user ít => latent vectors không ổn định  
- Ma trận đánh giá quá thưa => khó học được latent factors chính xác  
- Dữ liệu test chứa nhiều trường hợp gần như "cold start"

#### **2. Training RMSE là khá tốt**
Với thang điểm rating 1-5:

- RMSE = 1.13 nói rằng dự đoán trung bình chỉ lệch xấp xỉ 1 đơn vị  
- MAE = 0.6 cho thấy dự đoán trung bình sai khoảng 0.5 sao

=> Mô hình **nắm bắt được cấu trúc nội tại của dữ liệu**.

#### **3. MAE giảm mạnh cho thấy phân phối rating lệch về high values**
MAE tăng mạnh từ **0.6 đến 1.05** gợi ý:

- Nhiều rating trong test rơi vào các item mà user ít thông tin  
- MF không thể dự đoán chính xác trong các scenario sparse hoặc cold start  
- Test set có thể chứa các sản phẩm với rất ít rating lịch sử

### Kết luận:
- *Overfitting nhẹ + ảnh hưởng cold start* là nguyên nhân chính làm test error cao.  
- Tuy vậy, mức sai số test hiện tại vẫn **nằm trong khoảng hợp lý** đối với bài toán Amazon MF.  
- Mô hình đã hoạt động **ổn** cho một Matrix Factorization thuần NumPy.

*Lưu ý: Kết quả chạy mô hình Matrix Factorization có thể biến động nhẹ giữa các lần train do sử dụng SGD và dữ liệu Amazon Beauty có độ sparsity rất cao.*
